In [1]:
cd ~/durga/Experiments/loc_iparaphrasing/

/mnt/fs1/mayu-ot/Experiments/loc_iparaphrasing


In [2]:
import pandas as pd
from IPython.html.widgets import IntProgress
from IPython.display import display

/home/mayu-ot/miniconda3/envs/py36chainer/lib/python3.6/site-packages/IPython/html.py:14: ShimWarning: The `IPython.html` package has been deprecated since IPython 4.0. You should import from `notebook` instead. `IPython.html.widgets` has moved to `ipywidgets`.
  "`IPython.html.widgets` has moved to `ipywidgets`.", ShimWarning)


In [3]:
from nltk.metrics import edit_distance
import numpy as np
import json

In [4]:
def cleaning_df(df):
    # remove 'on' preposition
    replace_dict = {}
    for phrase in pd.unique(df.org_phrase):

        if phrase in ['on paper', 'in blue']:
            continue

        words  = phrase.split()
        if (words[0] == 'on') or (words[0] == 'in'):
            replace_dict[phrase] = ' '.join(words[1:])

    df['org_phrase'] = df.org_phrase.map(lambda x: replace_dict.get(x,x))
    return df

In [71]:
split = 'train'
pair_df = pd.read_csv('data/phrase_pair_%s.csv' % split, index_col=0)
bbox_df = pd.read_csv('data/pl-clc/localization_%s.csv' % split, index_col=0)
# bbox_df = cleaning_df(bbox_df)

/home/mayu-ot/miniconda3/envs/py36chainer/lib/python3.6/site-packages/numpy/lib/arraysetops.py:472: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [72]:
images = pd.unique(pair_df['image'])

ip = IntProgress(min=0, max=len(images))
display(ip)

# not_matched = []
# replace_dict = {}
replace = {'image':[], 'src_phr': [], 'tar_phr': []}

for c, i in enumerate(images):
    phrases = pd.unique(pair_df[pair_df.image == i][['original_phrase1', 'original_phrase2']].values.ravel())
    sub_bbox = bbox_df[bbox_df.image == i]['org_phrase']
    s_phrases = list(set(sub_bbox.values))
    for t_phr in phrases:
        if t_phr in s_phrases:
            continue
        else:
            dist = [edit_distance(x, t_phr) for x in s_phrases]
            s_phr = s_phrases[np.argmin(dist)]
            if t_phr == '" R':
                s_phr = '" " R'
            bbox_df.loc[(bbox_df.image == i) & (bbox_df.org_phrase == s_phr), 'org_phrase'] = t_phr
    ip.value = c

IntProgress(value=0, max=29769)

'other large blocks of concrete . An'

In [17]:
replace_dict = {}

for img, phr in not_matched:
    phrases = bbox_df.loc[bbox_df.image == img, 'org_phrase']
    phrases = list(set(phrases.values))
    dist = [edit_distance(phr, x) for x in phrases]
    t_phr = phrases[np.argmin(dist)]
    replace_dict[t_phr] = phr

if split == 'train':
    replace_dict['" " R'] = '" R' # exception

In [18]:
replace_dict

{"the ' thumbs up '": "the 'thumbs up '"}

In [19]:
json.dump(replace_dict, open('data/pl-clc/replace_dict_%s.json'%split, 'w'))

In [24]:
split = 'train'

df = pd.read_csv('data/pl-clc/localization_%s.csv' % split, index_col=0)

replace_dict = {}
for phrase in pd.unique(df.org_phrase):

    if phrase in ['on paper', 'in blue']:
        continue

    words  = phrase.split()
    if (words[0] == 'on') or (words[0] == 'in'):
        replace_dict[phrase] = ' '.join(words[1:])



# pcmp_rep_dict = json.load(open('data/pl-clc/replace_dict_%s.json'%split))
# replace_dict.update(pcmp_rep_dict)

# df['org_phrase'] = df.org_phrase.map(lambda x: replace_dict.get(x,x))

/home/mayu-ot/miniconda3/envs/py36chainer/lib/python3.6/site-packages/numpy/lib/arraysetops.py:472: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [25]:
replace_dict

{'in indian attire': 'indian attire',
 'in indigenous clothing': 'indigenous clothing',
 'in individual chairs': 'individual chairs',
 'in individual work spaces': 'individual work spaces',
 'in inflatable rafts': 'inflatable rafts',
 'in inflated boats': 'inflated boats',
 'in ink': 'ink',
 'in insects': 'insects',
 'in interesting attire': 'interesting attire',
 'in interesting clothes': 'interesting clothes',
 'in interesting wardrobe': 'interesting wardrobe',
 'in intricate quilt': 'intricate quilt',
 'in yellow': 'yellow',
 'in yellow boots': 'yellow boots',
 'in yellow overalls': 'yellow overalls',
 'on one': 'one',
 'on one 3 wheel motorcycle': 'one 3 wheel motorcycle',
 'on one arm': 'one arm',
 'on one black horse': 'one black horse',
 'on one blade': 'one blade',
 "on one boy 's head": "one boy 's head",
 'on one couch': 'one couch',
 'on one end': 'one end',
 'on one end of a rope': 'one end of a rope',
 'on one feet': 'one feet',
 'on one finger': 'one finger',
 'on one flo

In [26]:
pcmp_rep_dict = json.load(open('data/pl-clc/replace_dict_%s.json'%split))
replace_dict.update(pcmp_rep_dict)

# df['org_phrase'] = df.org_phrase.map(lambda x: replace_dict.get(x,x))

In [27]:
replace_dict

{'" " R': '" R',
 '" " full of bills': '" full of bills',
 '" " hurdles': '" hurdles',
 '" " jersey': '" jersey',
 '" C': '" R',
 '# 9': '#9',
 "' ' a ball": "' a ball",
 "' Nursing Idol": "'Nursing Idol",
 '8 people': '8+ people',
 'A . group of diverse people': 'A .group of diverse people',
 'A gas station name m & h': 'A gas station name m&h',
 'A women `': 'A women`',
 'Boy ’s camp drill Sergeant': 'Boy’s camp drill Sergeant',
 'C-U-B-S .': 'C-U-B-S.',
 "C.D 's": "C.D's",
 'CUBS .': 'CUBS.',
 'In India people': 'India people',
 'ROCKS .': 'ROCKS.',
 'Three yellow -': 'Three yellow-',
 'a # 51 boat': 'a #51 boat',
 "a ' NARVESEN ' sign": "a 'NARVESEN' sign",
 "a ' no child": "a 'no child",
 'a Lounge & Grill store': 'a Lounge&Grill store',
 'a Merry-go - round': 'a Merry-go -round',
 'a blue I < 3NY shirt': 'a blue I<3NY shirt',
 'a dumpster': 'off dumpster',
 'a fishing line': 'fishing poles',
 "a little tikes ' playground": "a little tikes' playground",
 "an ' evil clown": "an 'ev